<a href="https://colab.research.google.com/github/dhruv1710/AI-CuttingEdge/blob/main/Currency_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.0 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.56.0
    Uninstalling google-cloud-aiplatform-1.56.0:
      Successfully uninstalled google-cloud-aiplatform-1.56.0


In [ ]:
from google.colab import auth

In [ ]:
auth.authenticate_user()

In [ ]:
import vertexai
vertexai.init(project = 'encouraging-yen-427905-v6')
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information, including live exchange rates. 

To get the most up-to-date exchange rate for euros to dollars, I recommend checking a reliable online source such as:

* **Google Finance:** Search "EUR to USD" 
* **XE.com:** This website specializes in currency conversions.
* **Your Bank's Website:** Most banks provide current exchange rates for their customers.

Keep in mind that exchange rates fluctuate constantly, so the rate you see at any given moment may change. 



In [10]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content('''
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object as per the following example.
{{
  'from_currency': 'USD',
  'to_currency': 'EUR',
}}
'''.format(user_prompt=user_prompt))
print(response.text)

```json
{
  'from_currency': 'EUR',
  'to_currency': 'USD'
}
``` 



In [11]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-06-28","rates":{"AUD":1.6079,"BGN":1.9558,"BRL":5.8915,"CAD":1.467,"CHF":0.9634,"CNY":7.7748,"CZK":25.025,"DKK":7.4575,"GBP":0.84638,"HKD":8.3594,"HUF":395.1,"IDR":17487,"ILS":4.02,"INR":89.25,"ISK":148.9,"JPY":171.94,"KRW":1474.86,"MXN":19.5654,"MYR":5.0501,"NOK":11.3965,"NZD":1.7601,"PHP":62.56,"PLN":4.309,"RON":4.9773,"SEK":11.3595,"SGD":1.4513,"THB":39.319,"TRY":35.187,"USD":1.0705,"ZAR":19.497}}'

In [12]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

In [13]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [14]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [15]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [16]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'date': 'latest', 'from': 'AUD', 'to': 'SEK'}

In [17]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-06-28","rates":{"SEK":7.0648}}'

In [18]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'To calculate how much 500 Australian dollars is worth in Swedish krona, we need to get the latest exchange rate between these two currencies. \n\nAccording to the latest data (as of 2024-06-28), 1 Australian dollar (AUD) is equal to 7.0648 Swedish krona (SEK).\n\nTherefore, to calculate how much 500 AUD is worth in SEK, we can simply multiply the amount in AUD by the exchange rate:\n\n500 AUD * 7.0648 SEK/AUD = 3532.40 SEK\n\nTherefore, 500 Australian dollars are worth 3532.40 Swedish krona as of June 28, 2024. \n'